In [1]:
!pip install sqlalchemy 
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")
import logging

logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode="a"
    
)
import time



You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
engine=create_engine('sqlite:///inventory.db')

In [3]:
engine

Engine(sqlite:///inventory.db)

In [4]:
import os
import time
import pandas as pd
import logging
from sqlalchemy import create_engine

# Setup logging
logging.basicConfig(level=logging.INFO)

# Create engine (change this to your actual database)
engine = create_engine('sqlite:///inventory.db')  # Example for SQLite

def ingest_db(df, table_name, engine, if_exists='append'):
    df.to_sql(table_name, con=engine, if_exists=if_exists, index=False)

def load_raw_data():
    start = time.time()

    for file in os.listdir('data'):
        table_name = os.path.splitext(file)[0]
        file_path = os.path.join('data', file)

        if os.path.splitext(file)[1].lower() == '.csv':
            logging.info(f"Processing file: {file}")

            try:
                file_size = os.path.getsize(file_path)

                # If file is large, use chunking
                if file_size > 100 * 1024 * 1024:  # 100MB
                    chunksize = 100000
                    logging.info(f"Large file detected. Loading in chunks of {chunksize} rows.")
                    first_chunk = True
                    for chunk in pd.read_csv(file_path, chunksize=chunksize):
                        ingest_db(chunk, table_name, engine, if_exists='replace' if first_chunk else 'append')
                        first_chunk = False
                else:
                    df = pd.read_csv(file_path)
                    ingest_db(df, table_name, engine, if_exists='replace')

                logging.info(f"Finished ingesting: {file}")

            except Exception as e:
                logging.error(f"Error processing {file}: {e}")

    end = time.time()
    total_time = (end - start) / 60
    logging.info("Ingestion complete")
    logging.info(f"Total time taken: {total_time:.2f} minutes")


load_raw_data()



In [5]:
import os
import pandas as pd

print(os.path.splitext("example.csv"))  # Just test this line


('example', '.csv')


In [6]:
# def ingest_db(df,table_name,engine):#this function is built to ingest the dataframe to database table
#     df.to_sql(table_name,con=engine,if_exists='replace',index=False)